As noted in Tutorial 00, `pygsti` can provide many reports or presentation slides.

In this tutorial, we look at how to customize some of the output of `pygsti`'s report generating module.

In [1]:
#Import the GST module -- you probably want this at the beginning of every notebook
import pygsti
import json

Creating reports is a method of the `pygsti.report.Results` object. Below, we create such an object, and then initialize it with the appropriate fiducials, germs, maximum lengths, etc.

It's important to note that the `pygsti.report.Results` object requires an argument to indicate which objective function you wish to score the estimate using. It's recommended that, if you ran MC2GST, you use the `"chi2"` argument, while if you ran MLGST, you use `"logL"`. Otherwise, you will score your estimate based on an objective function you weren't optimizing over, so your scores may be rather weird!

Creating reports is a method of the `pygsti.report.Results` object. Below, we create such an object, and then initialize it with the appropriate fiducials, germs, maximum lengths, etc.

It's important to note that the `pygsti.report.Results` object requires an argument to indicate which objective function you wish to score the estimate using. It's recommended that, if you ran MC2GST, you use the `"chi2"` argument, while if you ran MLGST, you use `"logL"`. Otherwise, you will score your estimate based on an objective function you weren't optimizing over, so your scores may be rather weird!

First, we'll make smaller reports that don't contain any confidence intervals for the estimates,
nor any appendices.

NOTE: In order for you to create PowerPoint files, you'll have to have `python-pptx` installed on your system.

In [2]:
# Follow Algorithm tutorial to generate LSGST gatesets
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
gs_target.set_basis("gm",2)
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

#Get lists of gate strings for successive iterations of MC2GST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList]
        
gs_lsgst_list = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                         minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6),
                                         returnAll=True )

go_gatesets = [ pygsti.optimize_gauge(gs, "target", targetGateset=gs_target, gateWeight=1, spamWeight=0.001) for gs in gs_lsgst_list]

Loading from cache file:  tutorial_files/Example_Dataset.txt.cache

--- Iterative MC2GST: Beginning iter 1 of 10 : 92 gate strings ---
--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 48.2157 (92 data params - 40 model params = expected mean of 52; p-value = 0.623488)

--- Iterative MC2GST: Beginning iter 2 of 10 : 92 gate strings ---
--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)

--- Iterative MC2GST: Beginning iter 3 of 10 : 168 gate strings ---
--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 130.576 (168 data params - 40 model params = expected mean of 128; p-value = 0.420132)

--- Iterative MC2GST: Beginning iter 4 of 10 : 441 gate strings ---
--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 422.579 (441 data params - 40 model params = expected mean of 401; p-value = 0.220005)

--- Iterative MC2GST: Beginning iter 5 of 10 : 817 gate strings ---
--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 768.626 (817 data params - 40 m

In [3]:
res = pygsti.report.Results()
res.init_Ls_and_germs("chi2", gs_target, ds, gs_clgst, maxLengthList, germList,
                    go_gatesets, lsgstListOfLists, fiducialList, fiducialList, 
                    pygsti.construction.repeat_with_max_length, False)

In [4]:
#Make smaller reports (no confidence intervals or appendicies)
res.create_full_report_pdf(filename="tutorial_files/Example_reportB.pdf", verbosity=2, confidenceLevel=None,
                        debugAidsAppendix=False, gaugeOptAppendix=False,
                        pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_brief_report_pdf(filename="tutorial_files/Example_briefB.pdf", verbosity=2, confidenceLevel=None)

res.create_presentation_pdf(filename="tutorial_files/Example_slidesB.pdf", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_presentation_ppt(filename="tutorial_files/Example_slidesB.ppt", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

*** Generating tables ***
Generating table: targetSpamTable
Generating table: targetGatesTable
Generating table: datasetOverviewTable
Generating table: bestGatesetSpamTable
Generating table: bestGatesetSpamParametersTable
Generating table: bestGatesetGatesTable
Generating table: bestGatesetChoiTable
Generating table: bestGatesetDecompTable
Generating table: bestGatesetRotnAxisTable
Generating table: bestGatesetClosestUnitaryTable
Generating table: bestGatesetVsTargetTable
Generating table: bestGatesetErrorGenTable
Generating table: fiducialListTable
Generating table: prepStrListTable
Generating table: effectStrListTable
Generating table: germListTable
Generating table: progressTable
*** Generating plots ***
 -- Chi2 plots (2):  1  Generating figure: bestEstimateColorBoxPlot
2  Generating figure: invertedBestEstimateColorBoxPlot

*** Merging into template file ***
Latex file(s) successfully generated.  Attempting to compile with pdflatex...
Initial output PDF tutorial_files/Example_repo

In [5]:
res.create_full_report_pdf(filename="tutorial_files/Example_report.pdf", verbosity=2, confidenceLevel=95,
                        debugAidsAppendix=True, gaugeOptAppendix=True,
                        pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_brief_report_pdf(filename="tutorial_files/Example_brief.pdf", verbosity=2, confidenceLevel=95)

res.create_presentation_pdf(filename="tutorial_files/Example_slides.pdf", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_presentation_ppt(filename="tutorial_files/Example_slides.ppt", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

*** Generating tables ***
Generating table: targetSpamTable (w/95% CIs)
Generating table: targetGatesTable (w/95% CIs)
Generating table: datasetOverviewTable (w/95% CIs)
Generating table: bestGatesetSpamTable (w/95% CIs)
Generating table: bestGatesetSpamParametersTable (w/95% CIs)
Generating table: bestGatesetGatesTable (w/95% CIs)
Generating table: bestGatesetChoiTable (w/95% CIs)
Generating table: bestGatesetDecompTable (w/95% CIs)
Generating table: bestGatesetRotnAxisTable (w/95% CIs)
Generating table: bestGatesetClosestUnitaryTable (w/95% CIs)
Generating table: bestGatesetVsTargetTable (w/95% CIs)
Generating table: bestGatesetErrorGenTable (w/95% CIs)
Generating table: fiducialListTable (w/95% CIs)
Generating table: prepStrListTable (w/95% CIs)
Generating table: effectStrListTable (w/95% CIs)
Generating table: germListTable (w/95% CIs)
Generating table: progressTable (w/95% CIs)
Generating special: gaugeOptAppendixTables (w/95% CIs)
Generating special: gaugeOptAppendixTables
Genera

/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:414: RuntimeWarning: invalid value encountered in greater
  resdat[resdat>0] /= abs(vmax - midpoint)
/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:415: RuntimeWarning: invalid value encountered in less
  resdat[resdat<0] /= abs(vmin - midpoint)


In [6]:
import pygsti.report.latex as LU
reload(LU)
LU.latex_value(1.2e30,2)

'1\\e{30}'